In [16]:
# Install GDAL and Geopandas
!apt install gdal-bin python-gdal python3-gdal --quiet
!apt install python3-rtree --quiet
!pip install git+git://github.com/geopandas/geopandas.git --quiet
!pip install descartes --quiet
!pip install geopy
!pip install plotly_express
!pip install ipython-autotime

Reading package lists...
Building dependency tree...
Reading state information...
gdal-bin is already the newest version (2.2.3+dfsg-2).
python-gdal is already the newest version (2.2.3+dfsg-2).
python3-gdal is already the newest version (2.2.3+dfsg-2).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.
Reading package lists...
Building dependency tree...
Reading state information...
python3-rtree is already the newest version (0.8.3+ds-1).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.
time: 27.2 s (started: 2021-09-16 12:11:40 +00:00)


In [17]:
%load_ext autotime
import pandas as pd

import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

import matplotlib.pyplot as plt
import plotly_express as px
import tqdm
from tqdm import tqdm
from tqdm._tqdm_notebook import tqdm_notebook

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 19.3 ms (started: 2021-09-16 12:12:40 +00:00)


In [62]:
from google.colab import files
uploaded = files.upload()

Saving Federally_Recognized_Tribes.csv to Federally_Recognized_Tribes.csv
time: 9.84 s (started: 2021-09-16 13:13:41 +00:00)


In [63]:
import io

df = pd.read_csv(io.BytesIO(uploaded['Federally_Recognized_Tribes.csv']))
df.head()

,StateFullName,TribeFullName,Census Name,Tribe,TribeAlternateName,BIARegion,Directory,FWSRegion,USGSRegion,Latitude,Longtitude
0,ARIZONA,San Juan Southern Paiute Tribe of Arizona,NaN,San Juan,NaN,Western,Tribal,Region 2 - Southwest,Southwest,36.134993,-111.239863
1,VIRGINIA,Chickahominy Indian Tribe,Chickahominy_(state)_SDTSA,"Chickahominy Indian Tribe, Inc.",NaN,Eastern,NaN,Region 5 - Northeast,Northeast,37.413198,-77.124221
2,NEW YORK,Seneca Nation of Indians,NaN,Seneca,NaN,Eastern,Tribal,Region 5 - Northeast,Northeast,42.157841,-78.715031
3,UTAH,Confederated Tribes of the Goshute Reservation...,Goshute_Reservation,Confederated Goshute,NaN,Western,Tribal,Region 6 - Mountain-Prairie,Southwest,40.036604,-113.985280
4,ALASKA,Angoon Community Association,Angoon_ANVSA,Angoon,NaN,Alaska,Tribal,Region 7 - Alaska,Alaska,57.503300,-134.583900


time: 43.4 ms (started: 2021-09-16 13:13:53 +00:00)


In [64]:
df['Longtitude'] = df['Longtitude'].astype(str)
df['Latitude'] = df['Latitude'].astype(str)

df["Geom"] =  df['Latitude'].map(str)  + ',' + df['Longtitude'].map(str)
df.head()

,StateFullName,TribeFullName,Census Name,Tribe,TribeAlternateName,BIARegion,Directory,FWSRegion,USGSRegion,Latitude,Longtitude,Geom
0,ARIZONA,San Juan Southern Paiute Tribe of Arizona,NaN,San Juan,NaN,Western,Tribal,Region 2 - Southwest,Southwest,36.1349928,-111.2398625,"36.1349928,-111.2398625"
1,VIRGINIA,Chickahominy Indian Tribe,Chickahominy_(state)_SDTSA,"Chickahominy Indian Tribe, Inc.",NaN,Eastern,NaN,Region 5 - Northeast,Northeast,37.413198,-77.12422099999999,"37.413198,-77.12422099999999"
2,NEW YORK,Seneca Nation of Indians,NaN,Seneca,NaN,Eastern,Tribal,Region 5 - Northeast,Northeast,42.1578412,-78.7150311,"42.1578412,-78.7150311"
3,UTAH,Confederated Tribes of the Goshute Reservation...,Goshute_Reservation,Confederated Goshute,NaN,Western,Tribal,Region 6 - Mountain-Prairie,Southwest,40.036603899999996,-113.98528049999999,"40.036603899999996,-113.98528049999999"
4,ALASKA,Angoon Community Association,Angoon_ANVSA,Angoon,NaN,Alaska,Tribal,Region 7 - Alaska,Alaska,57.5033,-134.5839,"57.5033,-134.5839"


time: 43.7 ms (started: 2021-09-16 13:13:56 +00:00)


In [65]:
locator = Nominatim(user_agent="myGeocoder", timeout=10)
rgeocode = RateLimiter(locator.reverse, min_delay_seconds=0.001)
rgeocode

tqdm.pandas()
df['Address'] = df['Geom'].progress_apply(rgeocode)

100%|██████████| 588/588 [04:53<00:00,  2.00it/s]

time: 4min 53s (started: 2021-09-16 13:14:01 +00:00)


In [88]:
df.head()

# df[df['Address'].isna()]


,StateFullName,TribeFullName,Census Name,Tribe,TribeAlternateName,BIARegion,Directory,FWSRegion,USGSRegion,Latitude,Longtitude,Geom,Address
0,ARIZONA,San Juan Southern Paiute Tribe of Arizona,NaN,San Juan,NaN,Western,Tribal,Region 2 - Southwest,Southwest,36.1349928,-111.2398625,"36.1349928,-111.2398625","Cedar Avenue, Tuba City, Coconino County, Ariz..."
1,VIRGINIA,Chickahominy Indian Tribe,Chickahominy_(state)_SDTSA,"Chickahominy Indian Tribe, Inc.",NaN,Eastern,NaN,Region 5 - Northeast,Northeast,37.413198,-77.12422099999999,"37.413198,-77.12422099999999","8317, Samaria Lane, Charles City County, Virgi..."
2,NEW YORK,Seneca Nation of Indians,NaN,Seneca,NaN,Eastern,Tribal,Region 5 - Northeast,Northeast,42.1578412,-78.7150311,"42.1578412,-78.7150311","20, Wildwood Avenue, Salamanca, Cattaraugus Co..."
3,UTAH,Confederated Tribes of the Goshute Reservation...,Goshute_Reservation,Confederated Goshute,NaN,Western,Tribal,Region 6 - Mountain-Prairie,Southwest,40.036603899999996,-113.98528049999999,"40.036603899999996,-113.98528049999999","Ibapah School, Lincoln Highway, Ibapah, Tooele..."
4,ALASKA,Angoon Community Association,Angoon_ANVSA,Angoon,NaN,Alaska,Tribal,Region 7 - Alaska,Alaska,57.5033,-134.5839,"57.5033,-134.5839","167, Kootznahoo Road, Angoon, Unorganized Boro..."


time: 36.6 ms (started: 2021-09-16 13:35:28 +00:00)


In [109]:
def charDigit(word):
  for i in word:
    if i.isdigit():
      return True
    else:
      return False

time: 2.67 ms (started: 2021-09-16 14:23:55 +00:00)


In [132]:
df['Address'] = df['Address'].astype(str)

County = []
State = []
Country = []

for i in range(len(df)):
  lst = df['Address'][i].split(', ')
  print(lst)

  if len(lst) < 4:
    County.append(lst[-3])
    State.append(lst[-2])
    Country.append(lst[-1])
  
  elif len(lst) >=4 and not(charDigit(lst[-2])) and lst[-3] != 'Unorganized Borough':
    County.append(lst[-3])
    State.append(lst[-2])
    Country.append(lst[-1])
    
  elif len(lst) >=4 and not(charDigit(lst[-2])) and lst[-3] == 'Unorganized Borough':
    County.append(str(lst[-4] +', '+lst[-3]))
    State.append(lst[-2])
    Country.append(lst[-1])

  elif len(lst) >=4 and charDigit(lst[-2]) and lst[-4] == 'Unorganized Borough':
    County.append(str(lst[-5] +', '+lst[-4]))
    State.append(lst[-3])
    Country.append(lst[-1])

  elif len(lst) >=4 and charDigit(lst[-2]) and lst[-4] != 'Unorganized Borough':
    County.append(lst[-4])
    State.append(lst[-3])
    Country.append(lst[-1])

['Cedar Avenue', 'Tuba City', 'Coconino County', 'Arizona', '86045', 'United States']
['8317', 'Samaria Lane', 'Charles City County', 'Virginia', '23140', 'United States']
['20', 'Wildwood Avenue', 'Salamanca', 'Cattaraugus County', 'New York', '14779', 'United States']
['Ibapah School', 'Lincoln Highway', 'Ibapah', 'Tooele County', 'Utah', 'United States']
['167', 'Kootznahoo Road', 'Angoon', 'Unorganized Borough', 'Alaska', '99820', 'United States']
['Arctic Village', 'Unorganized Borough', 'Alaska', 'United States']
['Yukon Avenue', 'Chalkyitsik', 'Unorganized Borough', 'Alaska', 'United States']
['Chignik Lake', 'Lake and Peninsula', 'Alaska', 'United States']
['Nichols Street', 'Golovin', 'Nome', 'Alaska', 'United States']
['Steese Highway', 'Circle', 'Unorganized Borough', 'Alaska', 'United States']
['Iditarod National Historic Trail (Northern route)', 'Galena', 'Unorganized Borough', 'Alaska', '99741', 'United States']
['4th Avenue', 'Akhiok', 'Kodiak Island', 'Alaska', 'United 

In [126]:
df.insert(0, 'Country', Country)
df.insert(1, 'State', State)
df.insert(2, 'County', County)

time: 12.1 ms (started: 2021-09-16 14:44:48 +00:00)


In [128]:
# df.to_csv(r'D:/Ankur/Intership/AI4Rural/Tribal Nations/Federally_Recognized_Tribes_Cleaned.csv', index = False)
from google.colab import drive
drive.mount('drive')

Mounted at drive
time: 23.2 s (started: 2021-09-16 14:47:29 +00:00)


In [129]:
df.to_csv('Federally_Recognized_Tribes_Cleaned.csv')
!cp Federally_Recognized_Tribes_Cleaned.csv "drive/My Drive/Internship/AI4Rural"

time: 333 ms (started: 2021-09-16 14:49:02 +00:00)
